# Manipulação de Arquivos e Strings

### Semana 13 | Uso do Regex

## Questão Dirigida

Como gravar em um arquivo CSV os dados da pandemia de COVID-19 por país desta [página](https://pt.wikipedia.org/wiki/Portal:COVID-19) da Wikipedia?

## Desenvolvimento do Projeto

Para desenvolver este projeto, vamos utilizar técnicas de raspagem de dados na Web (Web Scraping), com a biblioteca `requests`, `BeautifulSoup` e expressões regulares.

In [1]:
# Só existe no computador do professor
# Vai dar erro no seu computador
from utils import funcs as fn

In [1]:
# download
import requests

# parsing do HTML
from bs4 import BeautifulSoup

# expressões regulares
import re

A incidência de COVID-19 por país pode ser encontrada na página da Wikipedia.

In [2]:
# URL de análise
url_pagina = 'https://pt.wikipedia.org/wiki/Portal:COVID-19'

In [3]:
# Utiliza a biblioteca requests para fazer o download da pagina HTML
pagina_html = requests.get(url_pagina)

# O Código 200 indica que o download foi bem sucedido
pagina_html

<Response [200]>

In [4]:
# O Beautiful Soup 
soup = BeautifulSoup(pagina_html.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Portal:COVID-19 – Wikipédia, a enciclopédia livre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"87fc2dba-9865-4292-9557-80f8a4fbcc5a","wgCSPNonce":!1,"wgCanonicalNamespace":"Portal","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":100,"wgPageName":"Portal:COVID-19","wgTitle":"COVID-19","wgCurRevisionId":58084687,"wgRevisionId":58084687,"wgArticleId":6223270,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Páginas com argumentos formatnum não numéricos","!CS1 francês-fontes em língua (fr)","!CS1 usa script na língua 

**Inspeção da página**

Inspecionando o código-fonte da página HTML no navegador (Chrome, Firefox etc) podemos ver que a tabela de interesse contém a classe CSS: `wikitable sortable mw-collapsible no-ref`. Vamos utilizar esta classe CSS para restringir a busca pelos elementos de interesse.

In [5]:
tb_covid_pais = soup.find(class_='wikitable sortable mw-collapsible no-ref')
tb_covid_pais

<table class="wikitable sortable mw-collapsible no-ref" style="text-align:center;">
<tbody><tr>
<th class="unsortable" colspan="5" style="white-space:nowrap; font-size:90%;"><div class="plainlinks hlist navbar mini" style="float:left; text-align:left"><ul><li class="nv-ver"><a href="/wiki/Predefini%C3%A7%C3%A3o:Dados_da_pandemia_de_COVID-19" title="Predefinição:Dados da pandemia de COVID-19"><abbr title="Ver esta predefinição">v</abbr></a></li><li class="nv-discutir"><a href="/wiki/Predefini%C3%A7%C3%A3o_Discuss%C3%A3o:Dados_da_pandemia_de_COVID-19" title="Predefinição Discussão:Dados da pandemia de COVID-19"><abbr title="Discutir esta predefinição">d</abbr></a></li><li class="nv-editar"><a class="external text" href="https://pt.wikipedia.org/w/index.php?title=Predefini%C3%A7%C3%A3o:Dados_da_pandemia_de_COVID-19&amp;action=edit"><abbr title="Editar esta predefinição">e</abbr></a></li></ul></div><div style="font-size:114%;margin:0 4em">Incidência da <a href="/wiki/Pandemia_de_COVID-19" 

Opcionalmente, pode-se fazer o seguinte.

In [ ]:
tbs_covid_pais = soup.find_all(class_='wikitable sortable mw-collapsible no-ref')
# Lembre-se que o método find_all retorna uma lista. No caso deste exemplo, 
# só existe um elemento na lista
tb_covid_pais = tbs_covid_pais[0]
# tb_covid_pais

In [6]:
# Retorna uma lista com todas as linhas da tabela
linhas_tb = tb_covid_pais.find_all('tr')
linhas_tb

[<tr>
 <th class="unsortable" colspan="5" style="white-space:nowrap; font-size:90%;"><div class="plainlinks hlist navbar mini" style="float:left; text-align:left"><ul><li class="nv-ver"><a href="/wiki/Predefini%C3%A7%C3%A3o:Dados_da_pandemia_de_COVID-19" title="Predefinição:Dados da pandemia de COVID-19"><abbr title="Ver esta predefinição">v</abbr></a></li><li class="nv-discutir"><a href="/wiki/Predefini%C3%A7%C3%A3o_Discuss%C3%A3o:Dados_da_pandemia_de_COVID-19" title="Predefinição Discussão:Dados da pandemia de COVID-19"><abbr title="Discutir esta predefinição">d</abbr></a></li><li class="nv-editar"><a class="external text" href="https://pt.wikipedia.org/w/index.php?title=Predefini%C3%A7%C3%A3o:Dados_da_pandemia_de_COVID-19&amp;action=edit"><abbr title="Editar esta predefinição">e</abbr></a></li></ul></div><div style="font-size:114%;margin:0 4em">Incidência da <a href="/wiki/Pandemia_de_COVID-19" title="Pandemia de COVID-19">pandemia</a> de <a href="/wiki/COVID-19" title="COVID-19">CO

**Investigando os resultados**

Brincando um pouco com esses objetos podemos ver que eles contém os dados que desejamos. No entanto, esses dados precisem ser tratados. Por exemplo: podemos ver que o resultado a seguir começa e termina com `\n` (caractere de nova linha); existe um espaço antes do nome do país; existem vários `\n` e `\xa0` (*line feed*) no meio da linha. Além disso, existem notas associadas ao nome do país e ao final da linha. Vamos cuidar de cada um desses problemas.

In [60]:
lin = linhas_tb[48].get_text()
lin

'\n Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]\n'

##### Removendo os `\n` e os espaços no começo e ao final da linha. 

O método `strip()` remove caracteres no começo e ao final da linha.

In [61]:
lin

'\n Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]\n'

In [10]:
lin.strip("\n")

' Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]'

In [11]:
lin.strip("\n").strip(" ")

'Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]'

Note que a variável `lin` não é alterada desta forma.

In [12]:
lin

'\n Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]\n'

Para que a variável `lin` seja alterada, é necessário utilizar a atribuição.

In [62]:
lin = lin.strip("\n").strip(" ")
lin

'Grécia\n404\xa0163\n\n12\xa0122\n\ndata-sort-value="−1"\n\n[62]'

##### Removendo os caracteres `\xa0` e `\n` no meio da string

Podemos utilizar o método `replace()` para substituir o caractere `\xa0` com a string vazia, para que o número de casos não esteja com espaços entre eles. Note que o caractere `\xa0` está em Unicode (que podem ser representados por hexadecimais), por isso necessitam do `u` antes do começo da string.

In [63]:
lin = lin.replace(u"\xa0", u"")
lin

'Grécia\n404163\n\n12122\n\ndata-sort-value="−1"\n\n[62]'

Como desejamos criar um arquivo CSV ao final do processo, vamos substituir cada `\n` que separa cada coluna da tabela por um `\t`.

In [64]:
lin = lin.replace("\n","\t")
lin

'Grécia\t404163\t\t12122\t\tdata-sort-value="−1"\t\t[62]'

##### Removendo as notas do nome do país e ao final da linha

Para remover as notas do nome do país, vamos utilizar a seguinte expressão regular: `r"\[.*\]"`. A expressão pode ser lida da seguinte forma: selecione o texto que começe com `[` contenha uma sequência qualquer de caracteres que termine com `]`. 

In [52]:
lin

'Estados Unidos[a]\t33432240\t\t600504\t\t–\t\t[2]'

In [65]:
lin = re.sub(r"\[[\d|a-z]+\]", "", lin)
lin

'Grécia\t404163\t\t12122\t\tdata-sort-value="−1"\t\t'

Remove-se finalmente o `\t` ao final da string.

In [66]:
lin = lin.rstrip("\t")
lin

'Grécia\t404163\t\t12122\t\tdata-sort-value="−1"'

In [67]:
lin = re.sub(r"\t\t", "\t", lin)
lin

'Grécia\t404163\t12122\tdata-sort-value="−1"'

In [68]:
lin = re.sub(r"\t–", "\t", lin)
lin

'Grécia\t404163\t12122\tdata-sort-value="−1"'

In [69]:
lin = re.sub(r"data-sort-value=.", "", lin)
lin

'Grécia\t404163\t12122\t−1"'

In [70]:
lin = re.sub(r"\"", "", lin)
lin

'Grécia\t404163\t12122\t−1'

Ao final da sequência (ex: Estados Unidos) deve ficar assim:

In [ ]:
lin = "Estados Unidos\t33416364\t600067\t"

##### Definição de uma função de preparação da linha

In [59]:
lin

'Estados Unidos\t33432240\t600504\t'

In [7]:
def prep_linha(linha):
    linha = linhas_tb[i].get_text()
    linha = linha.strip("\n").strip(" ")
    linha = linha.replace(u"\xa0", u"")
    linha = linha.replace("\n","\t")
    linha = re.sub(r"\[[\d|a-z]+\]", "", linha)
    linha = linha.rstrip("\t")
    linha = re.sub(r"\t\t", "\t", linha)
    linha = re.sub(r"–", "0", linha)
    linha = re.sub(r"data-sort-value=\"−1\"", "0", linha)
    linha = linha.replace("+", "")
    return linha

##### Descobrindo empiricamente onde a lista de países começa e termina

Brincando um pouco, podemos ver que a lista de arquivos começa na posição 2 de `linhas_tb` e termina na posição 225.

In [38]:
linh = linhas_tb[226].get_text()
linh

'\n Saba\n7\n\n0\n\n7\n\n[292]\n'

In [8]:
ini_tb = 2
fim_tb = 226

##### Preparando o texto do arquivo final

Para criarmos o texto arquivo CSV final, basta fazer uma concatenação de todas as linhas da tabela, incluindo um `\n` ao final da linha. 

In [9]:
# Esta primeira linha define o título da tabela
texto_final = "País\tCasos\tMortes\tCurados\n"
for i in range(ini_tb, fim_tb+1):
    linhas = prep_linha(linhas_tb[i])
    texto_final += linhas
    texto_final += "\n"
print(texto_final)

País	Casos	Mortes	Curados
Estados Unidos	33458134	601407	0
Índia	26752447	303720	23728011
Brasil	16720081	467706	15168330
França	5685915	109787	0
Turquia	5263697	47768	5131463
Rússia	5099182	122660	4711982
Reino Unido	4494699	127794	0
Itália	4223200	126283	3886867
Argentina	3852093	79320	3409253
Espanha	3687762	80049	0
Alemanha	3703807	89316	3498050
Colômbia	3459422	89808	3218620
Irã	2935443	80488	2494108
Polônia	2874092	74075	2642127
México	2423928	228146	1932457
Ucrânia	2209417	50959	2072021
Peru	1965432	69342	1918756
Chéquia	1662608	30136	1622876
África do Sul	1675013	56711	1565684
Países Baixos	1651780	17632	0
Indonésia	1 511 712	40 858	1 348 330
Canadá	1383214	25566	1326484
Chile	1394973	29385	1321600
Filipinas	1247899	21357	1170752
Iraque	1210105	16436	1125110
Romênia	1078338	30499	1041256
Suécia	1068473	14451	0
Bélgica	1065358	24982	0
Paquistão	926695	21022	852574
Portugal	850262	17026	810271
Israel	839508	6413	832693
Hungria	804382	29728	696029
Bangladesh	804293	12694	744065
Jo

## Gravando o texto em um arquivo

Para se gravar o texto em um arquivo, basta utilizar o código a seguir.

In [14]:
arquivo = open("lista-paises.tsv", "w", encoding= "utf-8")
arquivo.write(texto_final)
arquivo.close()

### Lendo o arquivo CSV com o Pandas

In [15]:
import pandas as pd
df = pd.read_csv("lista-paises.tsv", sep='\t')
df

,País,Casos,Mortes,Curados
0,Estados Unidos,33458134,601407,0
1,Índia,26752447,303720,23728011
2,Brasil,16720081,467706,15168330
3,França,5685915,109787,0
4,Turquia,5263697,47768,5131463
...,...,...,...,...
220,Santo Eustáquio,20,0,20
221,Montserrat,20,1,18
222,MS Zaandam,13,4,0
223,Coral Princess,12,3,0


In [16]:
df[df['País']=='Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1 511 712,40 858,1 348 330


In [19]:
df['Casos'] = df.Casos.apply(lambda celula: celula.replace(" ", ""))
df['Mortes'] = df.Mortes.apply(lambda celula: celula.replace(" ", ""))
df['Curados'] = df.Curados.apply(lambda celula: celula.replace(" ", ""))

In [20]:
df[df['País'] == 'Indonésia']

,País,Casos,Mortes,Curados
20,Indonésia,1511712,40858,1348330


In [21]:
df['Casos']=df.Casos.astype(int)
df['Mortes']=df.Mortes.astype(int)
df['Curados']=df.Curados.astype(int)

### Tarefas

1) Utilizando o plotly, faça a análise da evolução de Casos, Mortes e Curados do Dataset. 

In [42]:
import plotly.express as px

nm_coluna_x = "País"
nm_coluna_y = "Casos"
dfcasos_sorted = df.sort_values('Casos', ascending=False)

fig = px.bar(
    df,
    x = nm_coluna_x,
    y = nm_coluna_y,
    color_discrete_sequence=["red"], 
    height = 700,
    width = 800
)
fig.update_layout(
    title = "Quantidade de casos de Covid-19 por país",
    xaxis_title = "Países",
    yaxis_title = "Casos",
    width = 1000
)

fig.show()

In [43]:
nm_coluna_x = "País"
nm_coluna_y = "Mortes"
dfmortes_sorted = df.sort_values('Mortes', ascending=False)

fig = px.bar(
    dfmortes_sorted,
    x = nm_coluna_x,
    y = nm_coluna_y,
    color_discrete_sequence=["blue"], 
    height = 700,
    width = 800
)
fig.update_layout(
    title = "Quantidade de mortes por Covid-19 por país",
    xaxis_title = "Países",
    yaxis_title = "Mortes",
    width = 1000
)

fig.show()

In [44]:
nm_coluna_x = "País"
nm_coluna_y = "Curados"
dfcurados_sorted = df.sort_values('Curados', ascending=False)

fig = px.bar(
    dfcurados_sorted,
    x = nm_coluna_x,
    y = nm_coluna_y,
    color_discrete_sequence=["green"], 
    height = 700,
    width = 800
)
fig.update_layout(
    title = "Quantidade de curados da Covid-19 por país",
    xaxis_title = "Países",
    yaxis_title = "Curados",
    width = 1000
)

fig.show()

2) Calcule a estatística descritiva (média, soma, mediana e desvio-padrão) dos países do dataset.

In [54]:
print(round(df['Casos'].mean(), 3))    # Média do número de casos
print(df['Casos'].sum())               # Soma do número de casos
print(df['Casos'].mode())              # Moda do número de casos
print(df['Casos'].median())            # Mediana do número de casos
print(round(df['Casos'].std(), 3))     # Desvio padrão do número de casos

755399.573
169964904
0     20
1    128
dtype: int32
48594.0
3154204.262


In [55]:
print(round(df['Mortes'].mean(), 3))    # Média do número de mortes
print(df['Mortes'].sum())               # Soma do número de mortes
print(df['Mortes'].mode())              # Moda do número de mortes
print(df['Mortes'].median())            # Mediana do número de mortes
print(round(df['Mortes'].std(), 3))     # Desvio padrão do número de mortes

15757.604
3545461
0    0
dtype: int32
818.0
59533.086


In [53]:
print(df['Curados'].mean())              # Média do número de curados
print(df['Curados'].sum())               # Soma do número de curados
print(df['Curados'].mode())              # Moda do número de curados
print(df['Curados'].median())            # Mediana do número de curados
print(round(df['Curados'].std(), 3))     # Desvio padrão do número de curados

469537.0
105645825
0    0
dtype: int32
22118.0
1992353.136


3) Encontre uma tabela na Wikipedia do seu interesse e aplique as técnicas apresentadas neste caderno para criar um DataFrame, criar gráficos no Plotly e fazer as estatísticas descritivas.

In [2]:
url_campsp = 'https://pt.wikipedia.org/wiki/Campeonato_Paulista_de_Futebol'

In [4]:
# Utiliza a biblioteca requests para fazer o download da pagina HTML
pagina_html2 = requests.get(url_campsp)

# O Código 200 indica que o download foi bem sucedido
pagina_html2

<Response [200]>

In [6]:
# O Beautiful Soup 
soup_campsp = BeautifulSoup(pagina_html2.content, 'html.parser')
print(soup_campsp.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="pt">
 <head>
  <meta charset="utf-8"/>
  <title>
   Campeonato Paulista de Futebol – Wikipédia, a enciclopédia livre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro"],"wgRequestId":"3070dcde-a953-40ae-8d1d-679fd912c151","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Campeonato_Paulista_de_Futebol","wgTitle":"Campeonato Paulista de Futebol","wgCurRevisionId":61260747,"wgRevisionId":61260747,"wgArticleId":48485,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["!Páginas que usam Timeline","!Páginas com erros em referências","!Artig

In [10]:
table_camp = soup_campsp.find(class_='wikitable')
table_camp

<table class="wikitable" style="width:75%; font-size:90%">
<tbody><tr>
<th width="7%">Ano
</th>
<th width="7%"><span style="cursor: help; border-bottom: 1px dotted;" title="Organizador">Org.</span>
</th>
<th width="26%">Campeão
</th>
<th width="20%">Vice-campeão
</th>
<th width="20%">Terceiro colocado
</th>
<th width="20%">Quarto colocado
</th></tr>
<tr align="center" bgcolor="white">
<td><a href="/wiki/Campeonato_Paulista_de_Futebol_de_1902" title="Campeonato Paulista de Futebol de 1902">1902</a>
</td>
<td><small><a href="/wiki/Liga_Paulista_de_Foot-Ball" title="Liga Paulista de Foot-Ball">LPF</a></small>
</td>
<td><b><a href="/wiki/S%C3%A3o_Paulo_Athletic_Club" title="São Paulo Athletic Club">São Paulo Athletic</a></b> <small>(1)</small>
</td>
<td><a href="/wiki/Club_Athletico_Paulistano" title="Club Athletico Paulistano">Paulistano</a>
</td>
<td><a href="/wiki/Associa%C3%A7%C3%A3o_Atl%C3%A9tica_Mackenzie_College" title="Associação Atlética Mackenzie College">Mackenzie</a>
</td>
<td>

In [11]:
linhas_tab = table_camp.find_all('tr')
linhas_tab

[<tr>
 <th width="7%">Ano
 </th>
 <th width="7%"><span style="cursor: help; border-bottom: 1px dotted;" title="Organizador">Org.</span>
 </th>
 <th width="26%">Campeão
 </th>
 <th width="20%">Vice-campeão
 </th>
 <th width="20%">Terceiro colocado
 </th>
 <th width="20%">Quarto colocado
 </th></tr>,
 <tr align="center" bgcolor="white">
 <td><a href="/wiki/Campeonato_Paulista_de_Futebol_de_1902" title="Campeonato Paulista de Futebol de 1902">1902</a>
 </td>
 <td><small><a href="/wiki/Liga_Paulista_de_Foot-Ball" title="Liga Paulista de Foot-Ball">LPF</a></small>
 </td>
 <td><b><a href="/wiki/S%C3%A3o_Paulo_Athletic_Club" title="São Paulo Athletic Club">São Paulo Athletic</a></b> <small>(1)</small>
 </td>
 <td><a href="/wiki/Club_Athletico_Paulistano" title="Club Athletico Paulistano">Paulistano</a>
 </td>
 <td><a href="/wiki/Associa%C3%A7%C3%A3o_Atl%C3%A9tica_Mackenzie_College" title="Associação Atlética Mackenzie College">Mackenzie</a>
 </td>
 <td><a class="mw-redirect" href="/wiki/Sport

In [54]:
linha_ = linhas_tab[2].get_text()
linha_

'\n1903\n\nLPF\n\nSão Paulo Athletic(INV) (2)\n\nPaulistano\n\nMackenzie\n\nS.C. Internacional\n'

In [55]:
linha_ = linha_.strip('\n')
linha_

'1903\n\nLPF\n\nSão Paulo Athletic(INV) (2)\n\nPaulistano\n\nMackenzie\n\nS.C. Internacional'

In [56]:
linha_ = linha_.replace("\n","\t")
linha_

'1903\t\tLPF\t\tSão Paulo Athletic(INV) (2)\t\tPaulistano\t\tMackenzie\t\tS.C. Internacional'

In [57]:
linha_ = re.sub(r"\([\d|A-Z]+\)", "", linha_)
linha_

'1903\t\tLPF\t\tSão Paulo Athletic \t\tPaulistano\t\tMackenzie\t\tS.C. Internacional'

In [58]:
linha_ = re.sub(r"[ +]?\t\t", "\t", linha_)
linha_

'1903\tLPF\tSão Paulo Athletic\tPaulistano\tMackenzie\tS.C. Internacional'

In [52]:
teste_linha = linhas_tab[133].get_text()
teste_linha

'\n2021\n\nFPF\n\nSão Paulo (22)\n\nPalmeiras\n\nCorinthians\n\nMirassol\n'

In [33]:
i_tab = 1
f_tab = 133

In [36]:
def prep_tab_campsp(linhas):
    linhas = linhas_tab[i].get_text()
    linhas = linhas.strip('\n')
    linhas = linhas.replace("\n","\t")
    linhas = re.sub(r"\([\d|A-Z]+\)", "", linhas)
    linhas = re.sub(r"[ +]?\t\t", "\t", linhas)
    return linhas

In [37]:
tabela_final = "Ano\tOrganizador\tCampeão\tVice-campeão\tTerceiro colocado\tQuarto colocado\n"
for i in range(i_tab, f_tab+1):
    rows = prep_tab_campsp(linhas_tab[i])
    tabela_final += rows
    tabela_final += "\n"
print(tabela_final)

Ano	Organizador	Campeão	Vice-campeão	Terceiro colocado	Quarto colocado
1902	LPF	São Paulo Athletic	Paulistano	Mackenzie	Germânia
1903	LPF	São Paulo Athletic	Paulistano	Mackenzie	S.C. Internacional
1904	LPF	São Paulo Athletic	Paulistano	S.C. Internacional	Mackenzie
1905	LPF	Paulistano	Germânia	S.C. Internacional	São Paulo Athletic
1906	LPF	Germânia	S.C. Internacional	A.A. das Palmeiras	Paulistano
1907	LPF	S.C. Internacional	Paulistano  S.C. Americano		Germânia
1908	LPF	Paulistano	Germânia	S.C. Americano	S.C. Internacional
1909	LPF	A.A. das Palmeiras	Paulistano	S.C. Internacional	S.C. Americano
1910	LPF	A.A. das Palmeiras	S.C. Americano	São Paulo Athletic	Paulistano
1911	LPF	São Paulo Athletic	S.C. Americano	Paulistano	Ypiranga
1912	LPF	S.C. Americano	Paulistano	Germânia	Mackenzie
1913	APEA	Paulistano	Mackenzie	A.A. das Palmeiras	Não houve
LPF	S.C. Americano	Ypiranga	S.C. Internacional	Corinthians
1914	APEA	A.A. São Bento 	Paulistano	Mackenzie	Ypiranga
LPF	Corinthians	Campos Elíseos	Mina

In [42]:
arquivo = open("Tabela_Campeonato-Paulista.tsv", "w", encoding= "utf-8")
arquivo.write(tabela_final)
arquivo.close()

In [66]:
import pandas as pd
df2 = pd.read_csv("Tabela_Campeonato-Paulista.tsv", sep='\t')
df2

,Ano,Organizador,Campeão,Vice-campeão,Terceiro colocado,Quarto colocado
0,1902,LPF,São Paulo Athletic,Paulistano,Mackenzie,Germânia
1,1903,LPF,São Paulo Athletic,Paulistano,Mackenzie,S.C. Internacional
2,1904,LPF,São Paulo Athletic,Paulistano,S.C. Internacional,Mackenzie
3,1905,LPF,Paulistano,Germânia,S.C. Internacional,São Paulo Athletic
4,1906,LPF,Germânia,S.C. Internacional,A.A. das Palmeiras,Paulistano
...,...,...,...,...,...,...
128,2017,FPF,Corinthians,Ponte Preta,Palmeiras,São Paulo
129,2018,FPF,Corinthians,Palmeiras,São Paulo,Santos
130,2019,FPF,Corinthians,São Paulo,Palmeiras,Santos
131,2020,FPF,Palmeiras,Corinthians,Mirassol,Ponte Preta
